 # Mean Reciprocal Rank (MRR)

In [9]:
import pandas as pd
import seaborn as sns

palette = sns.color_palette(["#2176AB", "#F97662", "#FFBF00", "#50C878", "#B284BE"])
sns.set_palette(palette)


llm_res = pd.read_csv('../gym/results/llm_response.csv')
altair_res = pd.read_csv('../gym/results/altair_response.csv')

llm_res.drop(columns=['model', 'index', 'file_id', 'table_id'], inplace=True)
llm_res = llm_res.groupby(['id', 'query_id']).mean().reset_index()

altair_res = altair_res[(altair_res['threshold']*10).astype(int) == 0]
altair_res.drop(columns=['query', 'threshold', 'file_id', 'table_id'], inplace=True)

# get a df with all
df = pd.merge(llm_res, altair_res, on=['id', 'query_id'], how='inner')
df

,id,query_id,score,relevant,order
0,1609.05600v2_S5.T1,1,0.075000,0.00,0
1,1609.05600v2_S5.T1,2,0.050000,0.00,52
2,1609.05600v2_S5.T1,3,0.225000,0.25,14
3,1609.05600v2_S5.T1,4,0.250000,0.25,1
4,1609.05600v2_S5.T1,5,0.025000,0.00,70
...,...,...,...,...,...
1605,2410.13042v1_Sx1.T0a,6,0.442500,0.50,12
1606,2410.13042v1_Sx1.T0a,7,0.025000,0.00,155
1607,2410.13042v1_Sx1.T0a,8,0.033333,0.00,135
1608,2410.13042v1_Sx1.T0a,9,0.033333,0.00,154


In [10]:
# kendal correlation between df order and score
from scipy.stats import spearmanr

print(spearmanr(df['relevant'], df['score']))
print(spearmanr(df['order'], df['score']))


SignificanceResult(statistic=0.7539329228765741, pvalue=1.0343103417137328e-295)
SignificanceResult(statistic=-0.2327511187526052, pvalue=3.0187788851652297e-21)


In [11]:
scores = []

for i in range(1, 11):
    altair_table = altair_res[altair_res['query_id'] == i].sort_values('order')
    llm_table = llm_res[(llm_res['query_id'] == i) & (llm_res['relevant'] >= 0.2)]

    # get the first relevant document in altair_table
    # through the altair table to get the first relevant document in the llm table
    for j in range(len(altair_table)):
        if altair_table.iloc[j]['id'] in llm_table['id'].values:
            scores.append(1 / (j + 1))
            break

mrr = sum(scores) / len(scores)
print(f"MRR: {mrr}")

MRR: 0.6983333333333333
